# Transfer Learning Notebook for the Damped Harmonic Oscillator

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# imports
import torch
import matplotlib.pyplot as plt
import numpy as np
import sys
import json
import os
from scipy.integrate import solve_ivp

# Add parent directory to sys.path
from pathlib import Path
current_path = Path.cwd()
parent_dir = current_path.parent.parent
sys.path.append(str(parent_dir))

from src.transfer_learning import compute_H_and_dH_dt, analytically_compute_weights
from src.utils_plot import plot_loss_and_all_solution, plot_transfer_learned_and_analytical
from src.load_save import load_run_history

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [ ]:
def check_versions_and_device():
  # set the device to the GPU if it is available, otherwise use the CPU
  current_dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  !nvidia-smi
  return current_dev

# set a global device variable to use in code
dev = check_versions_and_device()
print(dev)

### Load the pretrain model and history

In [ ]:
file_name = "final_2081233"
equation_name = "DHO"

trained_model, \
x_range, \
iterations, \
hid_lay, \
num_equations, \
num_heads, \
loss_hist, \
alpha_list, \
A_list, \
IC_list, \
force_list = load_run_history(equation_name, file_name, dev, prev=False)

# function to get A for alpha value and DHO equation
def get_A(alpha):
    return torch.tensor([[0., -1.], [1., 2*alpha]], device=dev).double()

### Define numerical solution

In [ ]:
# Numerical solution
def double_coupled_equation(t, y, A, force):
    return np.array([force[0].detach().item() - A[0][1] * y[1] - A[0][0] * y[0],
                     force[1].detach().item() - A[1][0] * y[0] - A[1][1] * y[1]])

r_tol = 1e-4
numerical_sol_fct_radau = lambda x, v, A, force: (solve_ivp(double_coupled_equation, [x_range[0], x_range[1]],
                                                  v.squeeze(), args=(A, force), t_eval=x.squeeze(), method="Radau", rtol=r_tol).y)

numerical_sol_fct_rk45 = lambda x, v, A, force: (solve_ivp(double_coupled_equation, [x_range[0], x_range[1]],
                                                    v.squeeze(), args=(A, force), t_eval=x.squeeze(), method="RK45", rtol=r_tol).y)


numerical_methods = {"RK45": numerical_sol_fct_rk45, "Radau": numerical_sol_fct_radau}


### Plot training result

In [ ]:
plot_loss_and_all_solution(x_range=x_range, true_functs=numerical_sol_fct_radau,
                           trained_model=trained_model, IC_list=IC_list, A_list=A_list,
                           force=force_list, train_losses=loss_hist, device=dev)

## Transfer Learning

### Extract H 

In [ ]:
# forward pass to extract H
size = 512
H, H_0, dH_dt_new, t_eval = compute_H_and_dH_dt(x_range[0], x_range[1], trained_model, num_equations, hid_lay, size, dev)

### Transfer to bigger alpha

In [ ]:
# stiff parameter alpha list
alpha_transfer = 20

transfer_A = get_A(alpha_transfer)
force_transfer = force_list[0]
IC_transfer = IC_list[0]

# compute the transfer learned solution
M_inv_new, W_out_new_A, force_terms_new, total_time = analytically_compute_weights(dH_dt_new, H, H_0, t_eval,
                                                                       IC_transfer, transfer_A, force_transfer)

# plot the transfer learned and true solutions
plot_transfer_learned_and_analytical(H, W_out_new_A, t_eval, IC_transfer, transfer_A, force_transfer,
                                     num_equations, numerical_sol_fct_radau)

## Comparative analysis 

- Solve iteratively for several $\alpha$ value
- Solve with:
    - PINNS trasnfer learning (only A change)
    - RK45
    - Radeau

In [ ]:
import time
from collections import defaultdict
from tqdm import tqdm


alpha_list_transfer = [i for i in range(5, 80, 5)]
log_scale = False
#alpha_list_transfer = [i for i in range(2, 41, 2)]

computational_time = defaultdict(list)
max_error = defaultdict(list)
mean_error = defaultdict(list)

solution = defaultdict(list)

length = t_eval.shape[0]

for alpha in tqdm(alpha_list_transfer):

    transfer_A = get_A(alpha)
    force_transfer = force_list[0]
    IC_transfer = IC_list[0]

    # solve with PINNS
    M_inv_new, W_out_new_A, force_terms_new, total_time = analytically_compute_weights(dH_dt_new, H, H_0, t_eval,
                                                                                       IC_transfer, transfer_A,
                                                                                       force_transfer, verbose=False)
    pinns_sol = torch.matmul(H.double(), W_out_new_A.double())
    solution["PINNS"].append(np.swapaxes(pinns_sol.detach().cpu().numpy().squeeze(), 0, 1))
    computational_time["PINNS"].append(total_time)

    # solve with numerical methods
    for method, fct in numerical_methods.items():
        start = time.time()
        numerical_sol = fct(t_eval.detach().cpu().numpy(),
                            IC_transfer.detach().cpu(),
                            transfer_A.cpu(),
                            force_transfer.detach().cpu())
        solution[method].append(numerical_sol)
        end = time.time()
        computational_time[method].append(end-start)

### Plot the computational time

In [ ]:
color = {"PINNS": 'orange', "RK45": 'b', "Radau": 'g', 'LSODA': 'm', "True": (1, 0, 0, 0.5)}
fig, ax = plt.subplots(1, tight_layout=True, figsize=(13, 4))

for method, time in computational_time.items():

    ax.plot(alpha_list_transfer, time, "-o", color=color[method], label=f"{method}", linewidth=2 )

ax.set_title(r"Computational Time solving Equation with increasing Stiffness", fontsize=20)
ax.set_xlabel(r'Stiffness parameter ($\alpha$) and ratio ($SR$)', fontsize=16)
ax.set_ylabel('Time', fontsize=16)
ax.set_xticks(alpha_list_transfer[::2], [r"$\alpha$=" + str(2*i) + "\n" +rf"$SR$={4*i**2}" for i in alpha_list_transfer[::2]])
ax.grid()
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=16)
ax.legend(loc='best', fontsize=16)

### Plot MEA and MaxAE

In [ ]:
fig, ax = plt.subplots(1, tight_layout=True, figsize=(13, 4))

mae_y1 = np.abs(np.array(solution["PINNS"])-np.array(solution["Radau"])).mean(2)[:, 0]
ax.plot(alpha_list_transfer, mae_y1, "-o", label="$MAE$ ${y_1}$", linewidth=2, markersize=6)

mae_y2 = np.abs(np.array(solution["PINNS"])-np.array(solution["Radau"])).mean(2)[:, 1]
ax.plot(alpha_list_transfer, mae_y2, "-o", label="$MAE$ ${y_2}$", linewidth=2, markersize=6)

maxae_y1 = np.abs(np.array(solution["PINNS"])-np.array(solution["Radau"])).max(2)[:, 0]
ax.plot(alpha_list_transfer, maxae_y1, "-x", color="#1f77b4", label="$MaxAE$ ${y_1}$", linewidth=2, markersize=8)

maxae_y2 = np.abs(np.array(solution["PINNS"])-np.array(solution["Radau"])).max(2)[:, 1]
ax.plot(alpha_list_transfer, maxae_y2, "-x", color="#ff7f0e", label="MaxAE ${y_2}$", linewidth=2, markersize=8)

ax.set_yscale("log")
ax.set_title(r"Mean and Max Absolute Error with increasing Stiffness", fontsize=20)
ax.set_xlabel(r'Stiffness parameter $\alpha$ and ratio $SR$', fontsize=16)
ax.set_ylabel('Absolute Error', fontsize=16)
ax.set_xticks(alpha_list_transfer[::2], [r"$\alpha$=" + str(2*i) + "\n" +rf"$SR$={4*i**2}" for i in alpha_list_transfer[::2]])
ax.set_yticks([0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001],
              [r"$10^{-1}$", r"$10^{-2}$", r"$10^{-3}$", r"$10^{-4}$", r"$10^{-5}$", r"$10^{-6}$"])
ax.grid()
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=16)
ax.legend(loc='best', fontsize=14)

### Save MAE and MaxAE results over several alpha value

In [ ]:
history = {}
history["alpha_list"] = alpha_list
history["mae_y1"] = mae_y1
history["mae_y2"] = mae_y2
history["maxae_y1"] = maxae_y1
history["maxae_y2"] = maxae_y2

current_path = Path.cwd().parent.parent
path = os.path.join(current_path, "result_history")
with open(os.path.join(path, "DHO_Error_Transfer.json"),  "w") as fp:
    json.dump(history, fp)